In [3]:
!pip install pyBKT

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 32.7/32.7 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyBKT: filename=pyBKT-1.4-py3-none-any.whl size=34409 sha256=078716dbc9490c1821d90c575cfbd653b5080bb550ef1a4a1391e65628ba8bc7
  Stored in directory: c:\users\сергей\appdata\local\pip\cache\wheels\9a\d5\14\b0b1005880b40618565b8565b95bff41073b11a62919ff68b1
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=8484c1272315d1c55764bc06625dea62764f12e8c556e9d24d149265a15a11cc
  Stored in directory: c:\users\сергей\appdata\local\pip\cache\wheels\f8\e0\3d\9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built pyBKT sklearn


In [4]:
import numpy as np
import pandas as pd
from pyBKT.models import Model

In [6]:
ct_df = pd.read_table('data.txt', sep ='	')

C:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (61,63,65) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
print(f"Rows: {ct_df.shape[0]}")
print(f"Columns: {ct_df.shape[1]}")

Rows: 685615
Columns: 73


In [8]:
field = 'KC (WPI-Apr-2005)'
data = ct_df[ct_df[field].notna()]
data = ct_df[ct_df[field] == 'Fractions']

In [9]:
data = data[data['Outcome'] != 'HINT']

data.loc[data.Outcome == 'CORRECT', 'Outcome'] = 1
data.loc[data.Outcome == 'INCORRECT', 'Outcome'] = 0

print(data['Outcome'])

69        0
72        0
73        0
74        1
81        1
         ..
681222    0
681228    1
681771    1
684520    1
685457    1
Name: Outcome, Length: 3172, dtype: object


In [10]:
defaults = {'order_id': 'custom_order',
            'skill_name': 'custom_skill',
            'correct': 'custom_answer',
            'problem_id':'Problem Name'}

defaults['multiprior'] = 'custom_gs_classes'
columns = ['Row', 'Anon Student Id', 'KC (skills_from_dataframe)', 'Outcome', 'Level (Curriculum)', 'Problem Name']

custom_df = data[columns]
custom_df.columns = ['custom_order', 'Anon Student Id',
                     'custom_skill', 'custom_answer',
                     'custom_gs_classes',
                     'Problem Name']
custom_df.head(5)

,custom_order,Anon Student Id,custom_skill,custom_answer,custom_gs_classes,Problem Name
69,70,Stu_00176143fc2991346939888160a9b1ce,fractions,0,Assistment,2227
72,73,Stu_00176143fc2991346939888160a9b1ce,fractions,0,Assistment,2227
73,74,Stu_00176143fc2991346939888160a9b1ce,fractions,0,Assistment,2227
74,75,Stu_00176143fc2991346939888160a9b1ce,fractions,1,Assistment,2227
81,82,Stu_00176143fc2991346939888160a9b1ce,fractions,1,Assistment,2227


In [11]:
model = Model(seed = 41, num_fits = 1)
model.fit(data = custom_df, defaults = defaults)

In [12]:
def mae(true_vals, pred_vals):
    return np.mean(np.abs(true_vals - pred_vals))

In [13]:
#default model
training_rmse = model.evaluate(data = custom_df)
training_auc = model.evaluate(data = custom_df, metric = 'auc')
training_mae = model.evaluate(data = custom_df, metric = mae)

print(f"RMSE: {training_rmse:.5}")
print(f"AUC: {training_auc:.5}")
print(f"MAE: {training_mae:.5}")

preds = model.predict(data = custom_df)

RMSE: 0.47752
AUC: 0.64925
MAE: 0.46096


In [14]:
model_forget = Model(seed = 41, num_fits = 1)
model_forget.fit(data = custom_df, defaults = defaults, forgets=True)

In [15]:
#model with forgetting
training_rmse = model_forget.evaluate(data = custom_df)
training_auc = model_forget.evaluate(data = custom_df, metric = 'auc')
training_mae = model_forget.evaluate(data = custom_df, metric = mae)

print(f"RMSE: {training_rmse:.5}")
print(f"AUC: {training_auc:.5}")
print(f"MAE: {training_mae:.5}")

preds = model_forget.predict(data = custom_df)

RMSE: 0.47744
AUC: 0.65172
MAE: 0.46075


In [16]:
model_prior = Model(seed = 41, num_fits = 1)
model_prior.fit(data = custom_df, defaults = defaults,multiprior=True)

In [17]:
#multiprior model
training_rmse = model_prior.evaluate(data = custom_df)
training_auc = model_prior.evaluate(data = custom_df, metric = 'auc')
training_mae = model_prior.evaluate(data = custom_df, metric = mae)

print(f"RMSE: {training_rmse:.5}")
print(f"AUC: {training_auc:.5}")
print(f"MAE: {training_mae:.5}")

preds = model_prior.predict(data = custom_df)

RMSE: 0.47626
AUC: 0.65856
MAE: 0.61641
